<a href="https://colab.research.google.com/github/niyati-rao/Capstone_Project_LWE/blob/main/LWE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Using Sagemath
SageMath is not a pip-installable package. Please run this code locally after [installing sage](https://doc.sagemath.org/html/en/installation/index.html).

In [ ]:
import numpy as np
import math
from sage.all import GF

def mod(arr, b):
    res = arr % b
    return np.where(arr < 0, np.where(res == 0, 0, res - b), res)

In [ ]:
def nearestPrime(number):

    def getPrimes(limit):
        primes = []
        numbers = [True] * limit
        for i in range(2, limit):
            if numbers[i]:
                primes.append(i)
                for n in range(i ** 2, limit, i):
                    numbers[n] = False

        return primes

    primes = getPrimes(number + 100)
    maxDist = math.inf
    numb = 0

    for p in primes:
        if abs(number - p) < maxDist:
            maxDist = abs(number - p)
            numb = p

    return numb

In [ ]:
def solve_lwe(q, A, b, E):
    m = len(A)
    n = len(A[0])
    field = GF(q)
    pr = gf[tuple(f"x{i}" for i in range(n))]
    gens = pr.gens()

    f = []
    for i in range(m):
        p = 1
        for e in E:
            p *= (b[i] - sum(A[i][j] * gens[j] for j in range(n)) - e)
        f.append(p)

    s = []
    for p in pr.ideal(f).groebner_basis():
        assert p.nvariables() == 1 and p.degree() == 1
        s.append(int(-p.constant_coefficient()))

    return s

In [ ]:
def generate_lwe_instance(n, m, q, sigma, s=None):
    if s is None:
        s = np.random.randint(q, size=n)
    A = np.random.randint(q, size=(m, n))
    e = mod(np.round(np.random.normal(0, sigma, size=m)).astype(int), (q-1)/2) * (-1 if np.random.binomial(1,0.5)==1 else 1)
    b = (np.dot(A, s) + e) % q
    return A.tolist(), b.tolist(), s.tolist()

In [ ]:
n = 3
m = 5
sigma = pow(n,1/3)
q = nearestPrime(int(np.round(100*math.pow(sigma*math.log(n,2), 2))))

A, b, s_true = generate_lwe_instance(n, m, q, sigma)
E = [i for i in range(5)] + [i for i in range(0,-5,-1)]
print("True secret:", s_true)
rec = solve_lwe(q, A, b, E)
if (len(rec)!=0):
    print(rec)

Earlier attempt using SymPy (whose internal groebner computation is too slow)

In [ ]:
from sympy import symbols, Poly, solve, div, groebner, GF

def mod(arr, b):
    res = arr % b
    return np.where(arr < 0, np.where(res == 0, 0, res - b), res)

In [ ]:
gens = symbols(f"x0:{n}", integer=True)
p = Poly(1, *gens, modulus=q)
A, b, s_true = generate_lwe_instance(n, m, q, sigma)
p = p * Poly(b[0] - sum(A[0][j] * gens[j] for j in range(n)), *gens, modulus=q)
p * Poly(b[1] - sum(A[1][j] * gens[j] for j in range(n)), *gens, modulus=q)

Poly(-224*x0**2 - 250*x0*x1 + 68*x0*x2 - 232*x0 - 115*x1**2 + 59*x1*x2 + 134*x1 + 218*x2**2 - 231*x2 + 136, x0, x1, x2, modulus=523)

In [ ]:
def solve_lwe(q, A, b, E):
    m, n = len(A), len(A[0])
    gens = symbols(f"x0:{n}", integer=True)
    field = GF(q)
    equations = []
    for i in range(m):
        p = Poly(1, *gens, modulus=q)
        for e in E:
            p *= Poly(b[i] - sum(A[i][j] * gens[j] for j in range(n)) - e, *gens, modulus=q)
        equations.append(p)
    gb = groebner([p.as_expr() for p in equations], gens, modulus=q)
    sols = []
    for p in gb:
        if len(p.free_symbols) == 1:
            var = list(p.free_symbols)[0]
            coef = Poly(p, var, modulus=q).all_coeffs()
            if len(coef) == 2:
                sols.append((-coef[1] * pow(coef[0], -1, q)) % q)
    return sols


In [ ]:
def generate_lwe_instance(n, m, q, sigma, s=None):
    if s is None:
        s = np.random.randint(q, size=n)
    A = np.random.randint(q, size=(m, n))
    e = mod(np.round(np.random.normal(0, sigma, size=m)).astype(int), (q-1)/2) * (-1 if np.random.binomial(1,0.5)==1 else 1)
    b = (np.dot(A, s) + e) % q
    return A.tolist(), b.tolist(), s.tolist()

n = 3
sigma = pow(n,1/3)
q = nearestPrime(int(np.round(100*math.pow(sigma*math.log(n,2), 2))))

for m in range(5,6):
    A, b, s_true = generate_lwe_instance(n, m, q, sigma)
    E = [i for i in range(5)] + [i for i in range(0,-5,-1)]
    print("True secret:", s_true)
    rec = solve_lwe(q, A, b, E)
    if (len(rec)!=0):
        print(rec)

True secret: [259, 212, 421]


KeyboardInterrupt: 